In [ ]:
import os
import re
import sys
import time
import glob
import shutil
import requests
import subprocess
from urllib.parse import urlparse, unquote
from IPython.utils import capture
from IPython.display import clear_output, display, HTML

a2 = "--console-log-level=error -c -x 16 -s 16 -k 1M"
w = binascii.a2b_uu("%=V5B=6D`").decode('utf-8')
sd = binascii.a2b_uu("04W1A8FQE+61I9F9U<VEO;@``").decode('utf-8')
sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
sai = binascii.a2b_uu("=<W1A8FQE+61I9F9U<VEO;BUS=&%B:6QI='DM86D`").decode('utf-8')

# 忽略警告 ignore warnings
import warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"                    # 3：只显示ERROR和FATAL级别的日志消息，忽略所有其他级别的消息。
warnings.filterwarnings("ignore", category=UserWarning)     #这是一个警告过滤器，用于忽略UserWarning类别的警告。
!git config --global advice.detachedHead false              #禁用GIT在分离头状态（detached HEAD）下显示建议消息的功能
!sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

#检查是否使用GPU
def check_gpu():
    import tensorflow as tf
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("\r\033[96m GPU is available", flush=True)
    else:
        print("\r\033[91m 没有使用GPU,请在代码执行程序-更改运行时类型-设置为GPU!", flush=True)
        display(HTML("<img src='https://i.ibb.co/xfb7pB7/check-gpu.png' width='560px'/>"))
        from google.colab import runtime
        runtime.unassign()        
check_gpu()

#运行代码时,打印错误信息
def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

#下载自定义模型     #@param {type:"string"}
def download_model(xiazai_link,xiazai_path):
    urls = [url.strip() for url in re.split(r',|，', xiazai_link)]
    
    for url in urls:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            content_disposition = response.headers.get('content-disposition')
            if content_disposition:
                file_name = unquote(content_disposition.split('filename=')[1])
            else:
                file_name = unquote(url.split("/")[-1])
            
            filename = re.sub(r'[";]','', file_name)
            file_path = os.path.join(xiazai_path, file_name)

            if not os.path.exists(file_path):
                print(file_name, ",正在下载...")
                !aria2c {a2} {url} -o {file_name}
                print(file_name,",下载成功!")
            else:
                print(file_name,",文件已存在!" )
        else:
            print("下载链接错误:", url)



In [ ]:
#链接Google云盘
if sd_path == "drive" or use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')

%cd {sd_root}
print('\r\033[97m 正在克隆或更新项目...', flush=True)
if os.path.exists(sd_dir) and update_sd and sd_versions == "Automatic1111-latest":
    !git -C {sd_dir} reset --hard && git -C {sd_dir} checkout master && sleep 1 && rm {sd_dir}/webui.sh && git -C {sd_dir} pull
    print('项目更新完成！')
else:
    if sd_versions == "Automatic1111-latest":
        run_code(f"git clone -q --branch master https://github.com/AUTOMATIC1111/{sdw}")
    elif sd_versions == "Automatic1111-v1.5.2":
        run_code(f"git clone -q --branch v1.5.2 https://github.com/AUTOMATIC1111/{sdw}")
    print('项目克隆完成！')

print('\r\033[97m 正在安装SD运行环境...', flush=True)

run_code('apt-get -y update -qq')
run_code('apt-get -y install -qq aria2')

run_code("wget https://huggingface.co/Vanwise/sd-colab/resolve/main/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4")
os.environ['LD_PRELOAD'] = '/content/libtcmalloc_minimal.so.4'

if sd_versions == "Automatic1111-latest":
    with capture.capture_output() as cap:
        os.chdir('/content/')
        run_code("wget -q -i https://raw.githubusercontent.com/Van-wise/fast-sd/main/Dependencies/A1111.txt")
        os.system("dpkg -i *.deb")
        run_code('tar -C /content/ --zstd -xf sd_mrep.tar.zst')
        run_code('tar -C / --zstd -xf gcolabdeps.tar.zst')
        run_code('rm *.deb | rm *.zst | rm *.txt')
        run_code(f'mv -n /content/sd/stablediffusion/* {sd_dir}/repositories/{sai} && rm -rf /content/sd')
elif sd_versions == "Automatic1111-v1.5.2": 
    with capture.capture_output() as cap:
        run_code("pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U")
        run_code("pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U")
        run_code("pip install --prefer-binary https://huggingface.co/Vanwise/sd-colab/resolve/main/CLIP.zip")
        run_code("pip install --prefer-binary https://huggingface.co/Vanwise/sd-colab/resolve/main/open_clip.zip")
        run_code("wget https://huggingface.co/Vanwise/sd-colab/resolve/main/repositories.zip")
        run_code("unzip -q repositories.zip -d /root/main/repositories")
        run_code("pip install -r /root/main/requirements_versions.txt")

run_code('apt --fix-broken install')
run_code('pip install -qq pycloudflared ngrok tntn')
run_code('pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv pyfunctional')

print('运行环境，安装成功!')

os.makedirs(f"/content/{sd_root}/cache", exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = f"/content/{sd_root}/cache"
os.environ['TORCH_HOME'] = f"/content/{sd_root}/cache"

In [ ]:
#链接 ckpt
if ckpt_to_colab or sd_path == "colab":
    ckpt_dir = f"/root/{sd_name}/models/{sd}"
else:
    ckpt_dir = f"/content/drive/MyDrive/colab/models/{sd}"

print('\r\033[97m 正在下载主模型...', flush=True)
if ckpt1 == 'SDXL_base':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_base_1.0_0.9vae.safetensors'):
        !aria2c $a2 https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_base_1.0_0.9vae.safetensors
elif ckpt1 == 'RunDiffusionXL':
    if not os.path.exists(f'{ckpt_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c $a2 https://huggingface.co/ckpt/rundiffusion-xl/resolve/main/rundiffusionXL_beta.safetensors -d {ckpt_dir} -o rundiffusionXL_beta.safetensors
elif ckpt1 == 'JuggernautXL':
    if not os.path.exists(f'{ckpt_dir}/juggernautXL_version1.safetensors'):
        !aria2c $a2 https://huggingface.co/ckpt/juggernaut-xl/resolve/main/juggernautXL_version1.safetensors -d {ckpt_dir} -o juggernautXL_version1.safetensors
elif ckpt1 == '\u81EA\u5B9A\u4E49':
        !aria2c $a2 {ckpt1_downlink} -d {ckpt_dir}

if ckpt2 == 'SDXL_refiner':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_refiner_1.0.safetensors'):
        !aria2c $a2 https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_refiner_1.0.safetensors
elif ckpt2 == 'ChilloutMix':
        !aria2c $a2 https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {ckpt_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors
elif ckpt2 == 'Counterfeit':
        !aria2c $a2 https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp32.safetensors -d {ckpt_dir} -o Counterfeit-V3.0_fp32.safetensors
        
ckpt_custom = "https://civitai.com/models/25694/epicrealism\uFF0Chttps://civitai.com/api/download/models/161185"

if ckpt_downlink:
    def download_model(ckpt_downlink,ckpt_dir)   

#链接 lora
if lora_to_colab or sd_path == "colab":
    lora_dir = f"/content/{sd_name}/models/Lora"
else:
    lora_dir = f"/content/drive/MyDrive/colab/models/Lora"

print('\r\033[97m 正在下载loras模型...', flush=True)
if lora1 == 'detail-tweaker':
    if not os.path.exists(f'{lora_dir}/add_detail.safetensors'):
        !aria2c $a2 https://huggingface.co/Vanwise/sd-models/resolve/main/lora/add_detail.safetensors -d {lora_dir} -o add_detail.safetensors
elif lora1 == 'shojovibe_v11':
    if not os.path.exists(f'{lora_dir}/shojovibe_v11.safetensors'):
        !aria2c $a2 https://huggingface.co/Vanwise/sd-models/resolve/main/lora/shojovibe_v11.safetensors -d {lora_dir} -o shojovibe_v11.safetensors

if lora_downlink:
    def download_model(lora_downlink,lora_dir)   

#链接 Lycoris
lyco_dir = f"{sd_dir}/models/Lycoris"
print('\r\033[97m 正在下载lycoris模型...', flush=True)
if lyco1 == 'Neg4All':
    if not os.path.exists(f'{lyco_dir}/neg4all_bdsqlsz_V3.5.safetensors'):
        !aria2c $a2 https://huggingface.co/Vanwise/sd-models/resolve/main/lycoris/neg4all_bdsqlsz_V3.5.safetensors -d {lora_dir} -o neg4all_bdsqlsz_V3.5.safetensors
elif lyco1 == 'Neg4All_XL':
    if not os.path.exists(f'{lyco_dir}/neg4all_bdsqlsz_xl_V7.safetensors'):
        !aria2c $a2 https://huggingface.co/Vanwise/sd-models/resolve/main/lycoris/neg4all_bdsqlsz_xl_V7.safetensors -d {lora_dir} -o neg4all_bdsqlsz_xl_V7.safetensors 
elif lyco1 == 'EnvyBetterHands': 
    if not os.path.exists(f'{lyco_dir}/GoodHands-beta2.safetensors'):
        !aria2c $a2 https://civitai.com/api/download/models/55199 -d {lora_dir} -o GoodHands-beta2.safetensors

if lyco_downlink:
    def download_model(lyco_downlink,lyco_dir)   

# 链接 VAE
vae_dir = f"{sd_dir}/models/VAE"
print('\r\033[97m 正在下载vae模型...', flush=True)
if vae1 == 'kl-f8-anime2':
    if not os.path.exists(f'{vae_dir}/kl-f8-anime2.vae.pt'):
        !aria2c $a2 https://huggingface.co/Norisuke193/kl-f8-anime2/resolve/main/kl-f8-anime2.vae.pt -d {vae_dir} -o kl-f8-anime2.vae.pt
elif vae1 == 'vae-ft-mse-840000':
    if not os.path.exists(f'{vae_dir}/vae-ft-mse-840000-ema-pruned.safetensors'):
        !aria2c $a2 https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {vae_dir} -o vae-ft-mse-840000-ema-pruned.safetensors

if vae_downlink:
    def download_model(vae_downlink,vae_dir)  

# 链接 embeddings
embeddings_dir = f"{sd_dir}/models/embeddings"
print('\r\033[97m 正在下载embeddings模型...', flush=True)
if  dl_negative:
    !git clone https://huggingface.co/Vanwise/negative/resolve/main/negative.zip
    !unzip negative.zip -d {embeddings_dir}

if embeddings_downlink:
    def download_model(embeddings_downlink,embeddings_dir)  

# 链接 hypernetwork
hypernetworks_dir = f"{sd_dir}/models/hypernetworks" 
if hypernetworks_downlink:
    def download_model(hypernetworks_downlink,hypernetworks_dir)  

# 链接 scripts
scripts_dir = f"{sd_dir}/scripts" 
# 链接 clip_vision
clip_vision_dir = f"{sd_dir}/models/clip_vision"

# 链接 ControlNet
#@markdown ####<font color="#2b73af"> 选择ControlNet：
#@markdown #####<font color="#2c9678">🔅☑把ControlNet模型下载到colab；☐把模型下载到云盘。
cn_to_colab = True  #@param {type:"boolean"}

if cn_to_colab or sd_path == "colab":
    cn_dir = f"/content/{sd_name}/models/controlnet"
else:
    cn_dir = f"/content/drive/MyDrive/{sd_name}/models/controlnet"

def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: ' + os.path.basename(url))
        cmd = f"aria2c $a2 -d {model_dir} -o {filename} {url}"
        subprocess.run(cmd, shell=True, check=True)
    else:
        print(f"[1;32mThe model {filename} already exists[0m")

def cn_down():
    if os.path.exists("CN_models.txt"):
        with open("CN_models.txt", 'r') as f:
            mdllnk = f.read().splitlines()
    for lnk in mdllnk:
        download(lnk, cn_dir)

controlnet = "" # @param ["", "SD15_base", "SD15_full", "SDXL_base", "SDXL_full"]
if controlnet == "SD15_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15base.txt")
    cn_down()
elif controlnet == "SD15_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15full.txt")
    cn_down()
elif controlnet == "SDXL_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlbase.txt")
    cn_down()
elif controlnet == "SDXL_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlfull.txt")
    cn_down()
else:    
    print("不使用 ControlNet、或已使用快捷方式。")

In [ ]:
#@markdown ##<font color="#11659a"> MOD!
import pandas as pd
from google.colab import files, drive
#@markdown #####<font color="#2c9678">🔅☑启用MOD功能；☐不使用MOD。
enable_mod = False #@param {type:"boolean"} 
mod_name = "mod.xlsx"   #@param {type:"string"}
mod_link = ""   #@param {type:"string"} 

def get_mod():
    if mod_link:
        filename = os.path.basename(mod_link)
        if not filename.endswith('.xlsx'):
            print("下载链接错误!")
            return None
        !wget -q -T 10 {mod_link} -O {filename}
        mod_file = f"/content/{filename}"
        return mod_file 

    else:
        while True:
            try:
                uploaded = files.upload()
                if not uploaded:
                    mod_file = None
                    break
                mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])

                if not mod_file.endswith('.xlsx'):
                    raise ValueError("文件格式错误 .xlsx ")
                break

            except (ValueError, Exception) as e:
                print(e)
                print("请重新上传文件。")

        return mod_file

def use_mod():
    df = pd.read_excel(mod_file)

    for index, row in df.iterrows():
        file_type = row['类型']
        file_name = row['名称']
        download_link = row['下载地址']

        if file_type == 'CKPT':
            !aria2c $a2 {download_link} -d {ckpt_dir} -o {file_name}
        elif file_type == 'Lora':
            !aria2c $a2 {download_link} -d {lora_dir} -o {file_name}
        elif file_type == 'VAE':
            !aria2c $a2 {download_link} -d {vae_dir} -o {file_name}
        elif file_type == 'Extensions':
            repo_name = download_link.split('/')[-1]
            !git clone {download_link} {sd_dir}/extensions/{repo_name}
        elif file_type == 'Embeddings':
            !aria2c $a2 {download_link} -d {embeddings_dir} -o {file_name}
        elif file_type == 'Scripts':
            !aria2c $a2 {download_link} -d {sd_dir}/scripts -o {file_name}
        elif file_type == 'Controlnet':
            !aria2c $a2 {download_link} -d {cn_dir} -o {file_name}
        elif file_type == 'Comfyui_Nodes':
            nodes_name = download_link.split('/')[-1]
            !git clone {download_link} {sd_dir}/extensions/{nodes_name}

if enable_mod:
    mod_file = get_mod()
    if mod_file:
        use_mod()

In [ ]:
#@markdown---
#@markdown ####<font color="#2b73af"> 🌐☑使用云盘模型；☐不使用云盘模型。
use_drive_model = False  #@param {type:"boolean"}
#@markdown #####<font color="#2b73af"> 两种方法：cp（从云盘复制到colab）；ln（添加云盘软链接到colab）
content_way = "\u94FE\u63A5ln" #@param ["\u590D\u5236cp", "\u94FE\u63A5ln"]
if use_drive and use_drive_model:
    gckpt_dir = f"/content/drive/MyDrive/{sd_name}/models/checkpoints"
    gLora_dir = f"/content/drive/MyDrive/{sd_name}/models/Loras"
    gVAE_dir = f"/content/drive/MyDrive/{sd_name}/models/vae"
    gembeddings_dir = f"/content/drive/MyDrive/{sd_name}/embeddings"
    gcn_dir = f"/content/drive/MyDrive/{sd_name}/extensions/models/controlnet"
    goutputs_dir = f"/content/drive/MyDrive/{sd_name}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gcn_dir} {cn_dir}
    else:
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gcn_dir} {cn_dir}

#@markdown---
#@markdown ##<font color="#11659a">▶️ Start Fooocus
%cd {sd_dir}
!python launch.py --share